# Basketball-reference.com Web Scraper

The following code will pull and assemble a table of all the game stats of each player drafted into the NBA. Three separate tables will be created: A 'draft' table which summarizes all the players drafted and player specific information such as draft position, height, weight, and more. The second table will assemble all game logs found for the player found under international leagues and college basketball for all years prior to their draft year. The third table is all game stats for the player in the NBA. The datasets are not 100% inclusive due to some missing player info on basketball-reference.com. 

This code is flexible in that periods can be combined and not all the desired periods need to be run in the same session. Subsequent scrapings will be combined with previous scrapes and any overlapping years and or duplicates will be eliminated :) 

## Set file path and select years and then run all

In [15]:
path = 'C:/Users/erler/OneDrive/Documents/Random Data Sets/Basketball/'
#choose years span (though note that it runs extremely slow for many years at a time)
first_year = 2003
last_year = 2003
drafts_file_name = 'draft1'
preNBA_file_name = 'pre1'
gamelog_file_name = 'nba1'

### Import packages

In [16]:
#import required packages
import bs4 as bs
import urllib.request
import pandas as pd
import unicodedata
from itertools import cycle 
import re
import time
import numpy as np
import os
from datetime import datetime, timedelta 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

sauce = urllib.request.urlopen('https://www.basketball-reference.com/draft/NBA_2018.html').read()
soup = bs.BeautifulSoup(sauce, 'lxml')

def make_inches(x):
    try: 
        return float(x.split('-')[0])*12+float(x.split('-')[1])
    except:
        pass
    
def linkfix(x, testlink): #normalize links for website quirks
    if x[-1] != '/':
        x+='/'
    if x[:len(testlink)] == testlink:
        pass
    else:
        x=testlink+x
    return x

def minplayed(x): #convert minutes played into datetime format
    if ':' in x:
        a = pd.to_timedelta(float(x.split(':')[0]), unit= 'min')
        b = pd.to_timedelta(float(x.split(':')[1]), unit= 's')
        return a+b
    else:
        return pd.to_timedelta(float(x), unit='min')  

def ages(x): #convert age to float
        a = float(x.split('-')[0])
        b = float(x.split('-')[1])/365
        return a+b    

# Create table of all NBA picks for a given timespan

In [17]:
ticB1 = time.perf_counter()
years = [i for i in range(first_year, last_year+1)]
p1 = 'https://www.basketball-reference.com/draft/NBA_'
p2 = '.html'
drafts = []
for i in years:
    x = pd.read_html(p1+str(i)+p2)[0].droplevel(0, axis=1) #get each years draft class
    x['Year'] = i #add year to table
    x = x.loc[x.Rk.str.isnumeric()==True] #filter for headers and non-data
    x['pro_link']=np.nan #new
    sauce = urllib.request.urlopen(p1+str(i)+p2).read() #import html data
    soup = bs.BeautifulSoup(sauce, 'lxml') 
    table = soup.table #find table
    table_rows = table.find_all('tr') #find table rows
    for tr in table_rows: #find table links for players page and put in a list
        try:
            pick = int(tr.find_all('td')[0].get('csk')) #get pick number
        except:
            pass
        a = tr.find_all('a')
        for url in a: 
            if url.get('href')[:9] == '/players/':
                x.loc[x['Pk']==str(pick), 'pro_link'] = url.get('href')
                                
    drafts.append(x) #append each year
    
drafts = pd.concat(drafts)  #concat each df/year into one dataframe

#make a player name column that matches url use
drafts.Player = drafts.Player.replace({' ': '-'}, regex=True).str.lower() 
drafts['Player_url'] = [unicodedata.normalize('NFD', t).encode('ascii', 'ignore').decode('utf-8') for t in drafts.Player]
drafts.reset_index(drop=True, inplace=True)

tocB1 = time.perf_counter()
print(f"{tocB1 - ticB1:0.4f} seconds") #revised

0.8881 seconds


# Retrieve urls for college or international league gamelogs

In [18]:
ticB2 = time.perf_counter()

drafts['cbb_gamelogs'] = np.nan #set destination column
for j in range(len(drafts)+1): #for all players
    leagues = []
    seasons = []
    try:
        sauce = urllib.request.urlopen('https://www.basketball-reference.com'+drafts.pro_link[j]) #url main page of each player
        soup = bs.BeautifulSoup(sauce, 'lxml') 

        drafts.loc[j, 'Pro_height'] = soup.find_all('span',itemprop="height")[0].text
        drafts.loc[j, 'Pro_weight'] = re.sub("[^0-9]",'', soup.find_all('span',itemprop="weight")[0].text)
        drafts.loc[j, 'DOB'] = [i.get('data-birth') for i in soup.find_all('span') if i.get('data-birth')][0]

        a = soup.find_all('a') #find all links
        testlink='https://www.basketball-reference.com'  
        for url in a:
            try: #retrieve link to gamelogs for college games from pro profile
                if url.get('href')[:45] == 'https://www.sports-reference.com/cbb/players/':
                    linkfull = url.get('href')
                    sauce = urllib.request.urlopen(linkfull) #url main page of each cbb player
                    soup = bs.BeautifulSoup(sauce, 'lxml')
                    drafts.loc[j, 'cbb_height'] = soup.find_all('span',itemprop="height")[0].text #get cbb height
                    drafts.loc[j, 'cbb_weight'] = re.sub("[^0-9]",'', soup.find_all('span',itemprop="weight")[0].text) #get cbb weight
                    link = linkfull[:-5]+'/gamelog/' #adjust urls to get gamelogs
                    leagues.append(linkfix(link, 'https://www.sports-reference.com'))                     

                #retrieve links to gamelogs for international league games from pro profile
                if url.get('href')[:59] == 'https://www.basketball-reference.com/international/players/':
                    link = url.get('href') #get all links to intl play
                    sauce2 = urllib.request.urlopen(link) #get html
                    soup2 = bs.BeautifulSoup(sauce2, 'lxml') 
                    a2 = soup2.find_all('a') #find all links to leagues for each player from intl page
                    for url2 in a2: #get the links for gamelogs
                        try:
                            if 'gamelog' in url2.get('href'):
                                x = url2.get('href')
                                leagues.append(linkfix(x, testlink))

                        except:
                            pass

                #Retrieve NBA gamelogs        
                if 'gamelog' in url.get('href'):
                    x = url.get('href')
                    seasons.append(linkfix(x, testlink))   

            except:
                pass
    except:
        pass    
        
    leagues=set(leagues) #eliminate duplicates
    seasons=set(seasons) #eliminate duplicates
    drafts.loc[j, 'cbb_gamelogs'] = [leagues] #store in player table
    drafts.loc[j, 'pro_gamelogs'] = [seasons] #store in player table
    #print(j)
    
tocB2 = time.perf_counter()
print(f"{tocB2 - ticB2:0.4f} seconds")  #revised 

81.8851 seconds


In [5]:
#formating
drafts = drafts.drop([drafts.loc[drafts.Year.isnull()].index[0]], axis=0)

drafts.Pro_height = drafts.Pro_height.astype(str)   
drafts.Pro_height = drafts.Pro_height.apply(lambda x: make_inches(x))
drafts.loc[drafts.cbb_height.isna()!=True, 'cbb_height'] = drafts.loc[drafts.cbb_height.isna()!=True, 'cbb_height'].apply(lambda x: make_inches(x))
drafts.DOB = drafts.DOB.astype('datetime64')

drafts.Year = pd.to_datetime(drafts.Year, format='%Y') #dt format
drafts.Year = drafts.Year.apply(lambda x: x+pd.to_timedelta(180, unit='days')) #estimate draft date (used for cutoff purposes)

In [6]:
#save file
drafts.to_csv(path+drafts_file_name+str(first_year)+'_'+str(last_year)+'.csv', index=None) #save to folder in case failure
drafts=pd.DataFrame()
files = os.listdir(path)
files = [f for f in files if drafts_file_name in f]
for f in files:
    drafts_current = pd.read_csv(path+f)
    drafts = pd.concat([drafts, drafts_current])

drafts = drafts.drop_duplicates()
drafts.to_csv(path+drafts_file_name+'.csv', index=None)

# Assemble pre-NBA gamelogs into database

In [7]:
#if drafts table needs to be loaded and formatting needs to be restored
drafts = pd.read_csv(path+drafts_file_name+'.csv')
drafts['cbb_gamelogs'] = drafts['cbb_gamelogs'].apply(lambda x: eval(x))
drafts['pro_gamelogs'] = drafts['pro_gamelogs'].apply(lambda x: eval(x))
drafts.Year = pd.to_datetime(drafts.Year) #dt format
drafts.DOB = pd.to_datetime(drafts.DOB) #dt format

#limit data scrape to desired years range
drafts_iter = drafts.loc[(pd.DatetimeIndex(drafts.Year).year.astype(int)>=first_year)&(pd.DatetimeIndex(drafts.Year).year.astype(int)<=last_year)]

In [8]:
ticB3 = time.perf_counter()

stats = [] #destination list

for i in range(len(drafts_iter)+1):
    try:
        for q in drafts.loc[i, 'cbb_gamelogs']: #for each league
            x = pd.read_html(q)[0] #pull in gamelogs table
            x['League'] = 'NCAA' if q.split('/')[-2] == 'gamelog' else q.split('/')[-2] #league name
            #conform headers
            x.rename(columns={'School': 'Team', 'Unnamed: 7':'W/L', 'Unnamed: 5':'W/L', 'Unnamed: 4':'Location', 'Unnamed: 3':'Location'}, inplace=True)
            x['Player']=drafts.loc[i,'Player'] #player name
            x['DOB']=drafts.loc[i,'DOB'] #player name
            if x.Rk.dtype != 'O':        #remove junk
                x = x.loc[x.Rk.isna()!=True]
                stats.append(x)
            else:
                x = x.loc[x.Rk.str.isdigit()==True]
                stats.append(x)
    except:
        pass 
    #print(i)
Predraft_game_stats = pd.concat(stats) #combine dataframes

tocB3 = time.perf_counter()

print(f"{tocB3 - ticB3:0.4f} seconds")  #revised

259.6742 seconds


In [9]:
#Combine previous scrapes and current into one file 

#formatting
Predraft_game_stats.Date = pd.to_datetime(Predraft_game_stats.Date)
Predraft_game_stats.Rk = pd.to_numeric(Predraft_game_stats.Rk)
Predraft_game_stats[Predraft_game_stats.loc[:,'FG':'PTS'].columns.values] = Predraft_game_stats[Predraft_game_stats.loc[:,'FG':'PTS'].columns.values].astype('float64')
Predraft_game_stats = Predraft_game_stats.convert_dtypes(infer_objects=True)   

Predraft_game_stats['MP'] = Predraft_game_stats['MP'].astype(str)
Predraft_game_stats['MP'] = Predraft_game_stats['MP'].apply(lambda x: minplayed(x))
Predraft_game_stats.Date = Predraft_game_stats.Date.astype('datetime64')

Predraft_game_stats = pd.merge(Predraft_game_stats, drafts[['Player', 'DOB', 'Year']], how='left', left_on=['Player', 'DOB'], right_on=['Player', 'DOB'], copy=False, validate='m:m')
Predraft_game_stats['Year'] = pd.to_datetime(Predraft_game_stats.Year)

Predraft_game_stats = Predraft_game_stats.loc[Predraft_game_stats.Date<Predraft_game_stats.Year] #elimate intl stats after draft day

#force int64 to float64 for cooperation with statsmodel package
for i in Predraft_game_stats.columns:
    if Predraft_game_stats[i].dtype == 'Int64':
        Predraft_game_stats[i] = Predraft_game_stats[i].astype('float64')

C:\Users\erler\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


In [10]:
#save file
Predraft_game_stats.to_csv(path+preNBA_file_name+str(first_year)+'_'+str(last_year)+'.csv', index=None)
Predraft_game_stats = pd.DataFrame()
files = os.listdir(path)
files = [f for f in files if preNBA_file_name in f]
for f in files:
    Predraft_game_stats_current = pd.read_csv(path+f)
    Predraft_game_stats = pd.concat([Predraft_game_stats, Predraft_game_stats_current])

Predraft_game_stats = Predraft_game_stats.drop_duplicates()
Predraft_game_stats.to_csv(path+preNBA_file_name+'.csv', index=None)

# Assemble NBA gamelogs into database

In [11]:
#if drafts table needs to be loaded and formatting needs to be restored
drafts = pd.read_csv(path+drafts_file_name+'.csv')
drafts['cbb_gamelogs'] = drafts['cbb_gamelogs'].apply(lambda x: eval(x))
drafts['pro_gamelogs'] = drafts['pro_gamelogs'].apply(lambda x: eval(x))
drafts.Year = pd.to_datetime(drafts.Year) #dt format
drafts.DOB = pd.to_datetime(drafts.DOB) #dt format

#limit data scrape to desired years range
drafts_iter = drafts.loc[(pd.DatetimeIndex(drafts.Year).year.astype(int)>=first_year)&(pd.DatetimeIndex(drafts.Year).year.astype(int)<=last_year)]

In [12]:
ticB3 = time.perf_counter()

NBA_stats = [] #destination list

for i in range(len(drafts_iter)+1):
    try:
        for q in drafts.loc[i, 'pro_gamelogs']: #for each gamelog url
            x = pd.read_html(q)[7] #get table
            #conform column names
            x.rename(columns={'Unnamed: 7':'W/L', 'Unnamed: 8':'W/L', 'Unnamed: 5':'Location', 'Unnamed: 3':'Location'}, inplace=True)
            x.rename(columns=dict(zip(list(x.columns[x.columns.str.contains('Playoffs')==True]), cycle(['Date']))), inplace=True) 
            x['Player']=drafts.loc[i,'Player'] #get player name
            x['DOB']=drafts.loc[i,'DOB'] #get DOB
            if x.Rk.dtype != 'O':  #remove junk rows      
                x = x.loc[x.Rk.isna()!=True]
                NBA_stats.append(x)
            else:
                x = x.loc[x.Rk.str.isdigit()==True]
                NBA_stats.append(x)
    except:
        pass 
    #print(i)
NBA_game_stats = pd.concat(NBA_stats) #combine tables
NBA_game_stats = NBA_game_stats.loc[NBA_game_stats.MP.str.match(r'[^0-9]')!=True]
tocB3 = time.perf_counter()

print(f"{tocB3 - ticB3:0.4f} seconds")  #revised

147.6251 seconds


In [13]:
NBA_game_stats.drop(['Unnamed: 31'], axis=1, inplace=True) #remove unneeded cols
NBA_game_stats['MP'] = NBA_game_stats['MP'].apply(lambda x: minplayed(x)) 
NBA_game_stats.Date = NBA_game_stats.Date.astype('datetime64')

NBA_game_stats['DOB'] = NBA_game_stats['DOB'].astype('datetime64')    
NBA_game_stats.loc[NBA_game_stats['Age'].isna()!=True,'Age'] = NBA_game_stats.loc[NBA_game_stats['Age'].isna()!=True,'Age'].apply(lambda x: ages(x))
NBA_game_stats['Age'] = NBA_game_stats['Age'].astype(float)
NBA_game_stats = NBA_game_stats.merge(drafts[['Player', 'DOB', 'Year']], how = 'left', left_on=['Player', 'DOB'], right_on=['Player', 'DOB'], copy=False) #Bring in DOB (keep)
NBA_game_stats.loc[NBA_game_stats['Age'].isna()==True,'Age'] = ((NBA_game_stats.loc[NBA_game_stats.Age.isna()==True,'Date']-NBA_game_stats.loc[NBA_game_stats.Age.isna()==True,'DOB'])/np.timedelta64(1,'Y'))

NBA_game_stats['Years_pro'] = (NBA_game_stats.Date-NBA_game_stats.Year.astype('datetime64'))/np.timedelta64(1,'Y')
NBA_game_stats.Years_pro = NBA_game_stats.Years_pro - NBA_game_stats.Years_pro.min()

In [14]:
#save file
NBA_game_stats.to_csv(path+gamelog_file_name+str(first_year)+'_'+str(last_year)+'.csv', index=None) #save to folder in case failure
NBA_game_stats=pd.DataFrame()
files = os.listdir(path)
files = [f for f in files if gamelog_file_name in f]
for f in files:
    NBAp_current = pd.read_csv(path+f)
    NBA_game_stats = pd.concat([NBA_game_stats, NBAp_current])

NBA_game_stats = NBA_game_stats.drop_duplicates()
NBA_game_stats.to_csv(path+gamelog_file_name+'.csv', index=None)

C:\Users\erler\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
#Adjust draft year back to year now that only now that calculations are complete

drafts = pd.read_csv(path+drafts_file_name+'.csv')
drafts.Year = pd.to_datetime(drafts.Year)
drafts['Year'] = pd.DatetimeIndex(drafts.Year).year
drafts.to_csv(path+drafts_file_name+'.csv', index=None)

pre = pd.read_csv(path+preNBA_file_name+'.csv')
pre['Draft_year'] = pd.to_datetime(pre.Year)
pre['Draft_year'] = pd.DatetimeIndex(pre.Draft_year).year
pre = pre.drop(['Year'], axis=1)
pre.to_csv(path+preNBA_file_name+'.csv', index=None)

nba = pd.read_csv(path+gamelog_file_name+'.csv')
nba['Draft_year'] = pd.to_datetime(nba.Year)
nba['Draft_year'] = pd.DatetimeIndex(nba.Draft_year).year
nba = nba.drop(['Year'], axis=1)
nba.to_csv(path+gamelog_file_name+'.csv', index=None)